In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def get_sobel(in_chan, out_chan):
    filter_x = np.array([
        [1, 0, -1],
        [2, 0, -2],
        [1, 0, -1],
    ]).astype(np.float32)
    filter_y = np.array([
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1],
    ]).astype(np.float32)

    filter_x = filter_x.reshape((1, 1, 3, 3))
    filter_x = np.repeat(filter_x, in_chan, axis=1)
    filter_x = np.repeat(filter_x, out_chan, axis=0)

    filter_y = filter_y.reshape((1, 1, 3, 3))
    filter_y = np.repeat(filter_y, in_chan, axis=1)
    filter_y = np.repeat(filter_y, out_chan, axis=0)

    filter_x = torch.from_numpy(filter_x)
    filter_y = torch.from_numpy(filter_y)
    filter_x = nn.Parameter(filter_x, requires_grad=False)
    filter_y = nn.Parameter(filter_y, requires_grad=False)
    conv_x = nn.Conv2d(in_chan, out_chan, kernel_size=3, stride=1, padding=1, bias=False)
    conv_x.weight = filter_x
    conv_y = nn.Conv2d(in_chan, out_chan, kernel_size=3, stride=1, padding=1, bias=False)
    conv_y.weight = filter_y
    sobel_x = nn.Sequential(conv_x, nn.BatchNorm2d(out_chan))
    sobel_y = nn.Sequential(conv_y, nn.BatchNorm2d(out_chan))
    return sobel_x, sobel_y

def run_sobel(conv_x, conv_y, input):
    g_x = conv_x(input)
    g_y = conv_y(input)
    g = torch.sqrt(torch.pow(g_x, 2) + torch.pow(g_y, 2))
    return torch.sigmoid(g) * input

In [ ]:
!pip install torch torchvision
!pip3 install Cython

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import skimage.transform as skt
import shutil
import os



def get_sobel(in_chan, out_chan):
    filter_x = np.array([
        [1, 0, -1],
        [2, 0, -2],
        [1, 0, -1],
    ]).astype(np.float32)
    filter_y = np.array([
        [1, 2, 1],
        [0, 0, 0],
        [-1, -2, -1],
    ]).astype(np.float32)

    filter_x = filter_x.reshape((1, 1, 3, 3))
    filter_x = np.repeat(filter_x, in_chan, axis=1)
    filter_x = np.repeat(filter_x, out_chan, axis=0)

    filter_y = filter_y.reshape((1, 1, 3, 3))
    filter_y = np.repeat(filter_y, in_chan, axis=1)
    filter_y = np.repeat(filter_y, out_chan, axis=0)

    filter_x = torch.from_numpy(filter_x)
    filter_y = torch.from_numpy(filter_y)
    filter_x = nn.Parameter(filter_x, requires_grad=False)
    filter_y = nn.Parameter(filter_y, requires_grad=False)
    conv_x = nn.Conv2d(in_chan, out_chan, kernel_size=3, stride=1, padding=1, bias=False)
    conv_x.weight = filter_x
    conv_y = nn.Conv2d(in_chan, out_chan, kernel_size=3, stride=1, padding=1, bias=False)
    conv_y.weight = filter_y
    sobel_x = nn.Sequential(conv_x, nn.BatchNorm2d(out_chan))
    sobel_y = nn.Sequential(conv_y, nn.BatchNorm2d(out_chan))
    return sobel_x, sobel_y

def run_sobel(conv_x, conv_y, input):
    g_x = conv_x(input)
    g_y = conv_y(input)
    g = torch.sqrt(torch.pow(g_x, 2) + torch.pow(g_y, 2))
    return torch.sigmoid(g) * input






class BayarConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, padding=0):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.minus1 = (torch.ones(self.in_channels, self.out_channels, 1) * -1.000)

        super(BayarConv2d, self).__init__()
        # only (kernel_size ** 2 - 1) trainable params as the center element is always -1
        self.kernel = nn.Parameter(torch.rand(self.in_channels, self.out_channels, kernel_size ** 2 - 1),
                                   requires_grad=True)


    def bayarConstraint(self):
        self.kernel.data = self.kernel.permute(2, 0, 1)
        self.kernel.data = torch.div(self.kernel.data, self.kernel.data.sum(0))
        self.kernel.data = self.kernel.permute(1, 2, 0)
        ctr = self.kernel_size ** 2 // 2
        real_kernel = torch.cat((self.kernel[:, :, :ctr], self.minus1.to(self.kernel.device), self.kernel[:, :, ctr:]), dim=2)
        real_kernel = real_kernel.reshape((self.out_channels, self.in_channels, self.kernel_size, self.kernel_size))
        return real_kernel

    def forward(self, x):
        x = F.conv2d(x, self.bayarConstraint(), stride=self.stride, padding=self.padding)
        return x

def rgb2gray(rgb):
    b, g, r = rgb[:, 0, :, :], rgb[:, 1, :, :], rgb[:, 2, :, :]
    gray = 0.2989*r + 0.5870*g + 0.1140*b
    gray = torch.unsqueeze(gray, 1)
    return gray


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo
import torchvision
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt




model_urls = {
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
}

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, rate=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=rate, dilation=rate, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

# Define the ResNet model
class ResNet(nn.Module):
     def __init__(self, block, layers, num_classes=1000, n_input=3):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(n_input, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        rates = [1, 2, 4]
        self.layer4 = self._make_deeplabv3_layer(block, 512, layers[3], rates=rates, stride=1)  # stride 2 => stride 1
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

     def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

     def _make_deeplabv3_layer(self, block, planes, blocks, rates, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, rate=rates[i]))

        return nn.Sequential(*layers)

     def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet(pretrained=False, layers=[3,4,6,3], backbone='resnet50', n_input=3, **kwargs):
     """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
     model = ResNet(Bottleneck, layers, n_input=n_input, **kwargs)

     pretrain_dict = model_zoo.load_url(model_urls[backbone])
     try:
         model.load_state_dict(pretrain_dict,strict=False)
     except:
         print("loss conv1")
         model_dict = {}
         for k, v in pretrain_dict.items():
             if k in pretrain_dict and 'conv1' not in k:
                 model_dict[k] = v
         model.load_state_dict(model_dict, strict=False)
     print("load pretrain success")
     return model

class ResNet50(nn.Module):
    def __init__(self, pretrained=True,n_input=3):
        """Declare all needed layers."""
        super(ResNet50, self).__init__()
        self.model = resnet(n_input=n_input, pretrained=pretrained, layers=[3, 4, 6, 3], backbone='resnet50')
        self.relu = self.model.relu  # Place a hook

        layers_cfg = [4, 5, 6, 7]
        self.blocks = []
        for i, num_this_layer in enumerate(layers_cfg):
            self.blocks.append(list(self.model.children())[num_this_layer])

    def base_forward(self, x):
        feature_map = []
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)

        for i, block in enumerate(self.blocks):
            x = block(x)
            feature_map.append(x)

        out = nn.AvgPool2d(x.shape[2:])(x).view(x.shape[0], -1)

        return feature_map, out


class ERB(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ERB, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x, relu=True):
        x = self.conv1(x)
        res = self.conv2(x)
        res = self.bn(res)
        res = self.relu(res)
        res = self.conv3(res)
        if relu:
            return self.relu(x + res)
        else:
            return x + res

class MVSSNet(ResNet50):
    def __init__(self, nclass, aux=False, sobel=False, constrain=False, n_input=3, **kwargs):
        super(MVSSNet, self).__init__(pretrained=True, n_input=n_input)
        self.num_class = nclass
        self.aux = aux

        self.__setattr__('exclusive', ['head'])

        self.upsample = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.upsample_4 = nn.Upsample(scale_factor=4, mode="bilinear", align_corners=True)
        self.sobel = sobel
        self.constrain = constrain

        self.erb_db_1 = ERB(256, self.num_class)
        self.erb_db_2 = ERB(512, self.num_class)
        self.erb_db_3 = ERB(1024, self.num_class)
        self.erb_db_4 = ERB(2048, self.num_class)

        self.erb_trans_1 = ERB(self.num_class, self.num_class)
        self.erb_trans_2 = ERB(self.num_class, self.num_class)
        self.erb_trans_3 = ERB(self.num_class, self.num_class)

        if self.sobel:
            print("----------use sobel-------------")
            self.sobel_x1, self.sobel_y1 = get_sobel(256, 1)
            self.sobel_x2, self.sobel_y2 = get_sobel(512, 1)
            self.sobel_x3, self.sobel_y3 = get_sobel(1024, 1)
            self.sobel_x4, self.sobel_y4 = get_sobel(2048, 1)

        if self.constrain:
            print("----------use constrain-------------")
            self.noise_extractor = ResNet50(n_input=3, pretrained=True)
            self.constrain_conv = BayarConv2d(in_channels=1, out_channels=3, padding=2)
            self.head = _DAHead(2048+2048, self.num_class, aux, **kwargs)
        else:
            self.head = _DAHead(2048, self.num_class, aux, **kwargs)

    def forward(self, x):
        print('actual shape check:',x.shape)
        print('device of x:',x.device)
        size = x.size()[2:]
        input_ = x.clone()
        feature_map, _ = self.base_forward(input_)
        c1, c2, c3, c4 = feature_map

        if self.sobel:
            res1 = self.erb_db_1(run_sobel(self.sobel_x1, self.sobel_y1, c1))
            res1 = self.erb_trans_1(res1 + self.upsample(self.erb_db_2(run_sobel(self.sobel_x2, self.sobel_y2, c2))))
            res1 = self.erb_trans_2(res1 + self.upsample_4(self.erb_db_3(run_sobel(self.sobel_x3, self.sobel_y3, c3))))
            res1 = self.erb_trans_3(res1 + self.upsample_4(self.erb_db_4(run_sobel(self.sobel_x4, self.sobel_y4, c4))), relu=False)

        else:
            res1 = self.erb_db_1(c1)
            res1 = self.erb_trans_1(res1 + self.upsample(self.erb_db_2(c2)))
            res1 = self.erb_trans_2(res1 + self.upsample_4(self.erb_db_3(c3)))
            res1 = self.erb_trans_3(res1 + self.upsample_4(self.erb_db_4(c4)), relu=False)

        if self.constrain:
            x = rgb2gray(x)
            x = self.constrain_conv(x)
            constrain_features, _ = self.noise_extractor.base_forward(x)
            constrain_feature = constrain_features[-1]
            c4 = torch.cat([c4, constrain_feature], dim=1)

        outputs = []

        x = self.head(c4)
        x0 = F.interpolate(x[0], size, mode='bilinear', align_corners=True)
        outputs.append(x0)

        if self.aux:
            x1 = F.interpolate(x[1], size, mode='bilinear', align_corners=True)
            x2 = F.interpolate(x[2], size, mode='bilinear', align_corners=True)
            outputs.append(x1)
            outputs.append(x2)

        print(res1.shape)
        print(x0.shape)

        return res1, x0


class _PositionAttentionModule(nn.Module):
    """ Position attention module"""

    def __init__(self, in_channels, **kwargs):
        super(_PositionAttentionModule, self).__init__()
        self.conv_b = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.conv_c = nn.Conv2d(in_channels, in_channels // 8, 1)
        self.conv_d = nn.Conv2d(in_channels, in_channels, 1)
        self.alpha = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size, _, height, width = x.size()
        feat_b = self.conv_b(x).view(batch_size, -1, height * width).permute(0, 2, 1)
        feat_c = self.conv_c(x).view(batch_size, -1, height * width)
        attention_s = self.softmax(torch.bmm(feat_b, feat_c))
        feat_d = self.conv_d(x).view(batch_size, -1, height * width)
        feat_e = torch.bmm(feat_d, attention_s.permute(0, 2, 1)).view(batch_size, -1, height, width)
        out = self.alpha * feat_e + x

        return out


class _ChannelAttentionModule(nn.Module):
    """Channel attention module"""

    def __init__(self, **kwargs):
        super(_ChannelAttentionModule, self).__init__()
        self.beta = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        batch_size, _, height, width = x.size()
        feat_a = x.view(batch_size, -1, height * width)
        feat_a_transpose = x.view(batch_size, -1, height * width).permute(0, 2, 1)
        attention = torch.bmm(feat_a, feat_a_transpose)
        attention_new = torch.max(attention, dim=-1, keepdim=True)[0].expand_as(attention) - attention
        attention = self.softmax(attention_new)

        feat_e = torch.bmm(attention, feat_a).view(batch_size, -1, height, width)
        out = self.beta * feat_e + x

        return out


class _DAHead(nn.Module):
    def __init__(self, in_channels, nclass, aux=True, norm_layer=nn.BatchNorm2d, norm_kwargs=None, **kwargs):
        super(_DAHead, self).__init__()
        self.aux = aux
        inter_channels = in_channels // 4
        self.conv_p1 = nn.Sequential(
            nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
            norm_layer(inter_channels, **({} if norm_kwargs is None else norm_kwargs)),
            nn.ReLU(True)
        )
        self.conv_c1 = nn.Sequential(
            nn.Conv2d(in_channels, inter_channels, 3, padding=1, bias=False),
            norm_layer(inter_channels, **({} if norm_kwargs is None else norm_kwargs)),
            nn.ReLU(True)
        )
        self.pam = _PositionAttentionModule(inter_channels, **kwargs)
        self.cam = _ChannelAttentionModule(**kwargs)
        self.conv_p2 = nn.Sequential(
            nn.Conv2d(inter_channels, inter_channels, 3, padding=1, bias=False),
            norm_layer(inter_channels, **({} if norm_kwargs is None else norm_kwargs)),
            nn.ReLU(True)
        )
        self.conv_c2 = nn.Sequential(
            nn.Conv2d(inter_channels, inter_channels, 3, padding=1, bias=False),
            norm_layer(inter_channels, **({} if norm_kwargs is None else norm_kwargs)),
            nn.ReLU(True)
        )
        self.out = nn.Sequential(
            nn.Dropout(0.1),
            nn.Conv2d(inter_channels, nclass, 1)
        )
        if aux:
            self.conv_p3 = nn.Sequential(
                nn.Dropout(0.1),
                nn.Conv2d(inter_channels, nclass, 1)
            )
            self.conv_c3 = nn.Sequential(
                nn.Dropout(0.1),
                nn.Conv2d(inter_channels, nclass, 1)
            )

    def forward(self, x):
        feat_p = self.conv_p1(x)
        feat_p = self.pam(feat_p)
        feat_p = self.conv_p2(feat_p)

        feat_c = self.conv_c1(x)
        feat_c = self.cam(feat_c)
        feat_c = self.conv_c2(feat_c)

        feat_fusion = feat_p + feat_c

        outputs = []
        fusion_out = self.out(feat_fusion)
        outputs.append(fusion_out)
        if self.aux:
            p_out = self.conv_p3(feat_p)
            c_out = self.conv_c3(feat_c)
            outputs.append(p_out)
            outputs.append(c_out)

        return tuple(outputs)


def get_mvss(backbone='resnet50', pretrained_base=True, nclass=1, sobel=True, n_input=3, constrain=True, **kwargs):
    model = MVSSNet(nclass, backbone=backbone,
                    pretrained_base=pretrained_base,
                    sobel=sobel,
                    n_input=n_input,
                    constrain=constrain,
                    **kwargs)
    return model

In [ ]:
x = torch.randn([1,3,256,256])

In [ ]:
model = get_mvss()
out = model(x)
print(out)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 176MB/s]


load pretrain success
----------use sobel-------------
----------use constrain-------------
load pretrain success
actual shape check: torch.Size([1, 3, 256, 256])
device of x: cpu
torch.Size([1, 1, 64, 64])
torch.Size([1, 1, 256, 256])
(tensor([[[[ 1.1001,  1.2593,  1.2432,  ...,  1.1124,  0.8553,  0.5751],
          [ 0.9826,  0.6777,  0.4448,  ...,  0.5605,  0.3095,  0.5068],
          [ 0.8470,  0.6108,  0.2798,  ...,  0.4275,  0.2531,  0.4747],
          ...,
          [ 1.5506,  0.5496,  0.4395,  ...,  0.8315, -0.0549,  0.8615],
          [ 2.1656,  2.2038,  0.9866,  ...,  0.8629, -0.3035,  0.3360],
          [-1.3258,  1.2109,  0.2265,  ...,  0.1600,  0.1871,  0.2449]]]],
       grad_fn=<AddBackward0>), tensor([[[[-0.5448, -0.4801, -0.4154,  ...,  0.2655,  0.2489,  0.2323],
          [-0.5611, -0.4992, -0.4374,  ...,  0.2943,  0.2813,  0.2683],
          [-0.5773, -0.5184, -0.4594,  ...,  0.3232,  0.3137,  0.3043],
          ...,
          [ 0.1973,  0.1892,  0.1812,  ..., -0.066

#Data Loading Task

In [ ]:
dataset_path = '/content/drive/MyDrive/dataset'

#image_dir = "/content/drive/MyDrive/dataset/COVERAGE/train/images"
#image_names = os.listdir(image_dir)

# Path to the COVERAGE folder
coverage_path = os.path.join(dataset_path, 'COVERAGE')

# Path to the train and valid folders
train_path = os.path.join(coverage_path, 'train')
valid_path = os.path.join(coverage_path, 'valid')

# Function to load and preprocess the images
#import torch

def load_images(path):
    image_list = []
    for file in os.listdir(path):
        if file.endswith('.jpg'):
            image_path = os.path.join(path, file)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (256, 256))  # Reshape image to 256x256
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_list.append(image)
    return np.array(image_list)


# Load and preprocess the training images
train_images_path = os.path.join(train_path, 'images')
train_images = load_images(train_images_path)
train_images = torch.from_numpy(train_images).permute(0, 3, 1, 2).float() / 255.0  # Convert to tensor, adjust shape and normalize

# Load and preprocess the validation images
valid_images_path = os.path.join(valid_path, 'images')
valid_images = load_images(valid_images_path)
valid_images = torch.from_numpy(valid_images).permute(0, 3, 1, 2).float() / 255.0  # Convert to tensor, adjust shape and normalize

train_masks_path = os.path.join(train_path, 'masks')
train_masks = load_images(train_masks_path)
train_masks = torch.from_numpy(train_masks).permute(0, 3, 1, 2).float() / 255.0  # Convert to tensor, adjust shape and normalize

# Load and preprocess the validation mask images
valid_masks_path = os.path.join(valid_path, 'masks')
valid_masks = load_images(valid_masks_path)
valid_masks = torch.from_numpy(valid_masks).permute(0, 3, 1, 2).float() / 255.0  # Convert to tensor, adjust shape and normalize




# Calculate the mean values of training images and masks
train_images_mean = torch.mean(train_images, dim=(0, 2, 3))
train_masks_mean = torch.mean(train_masks, dim=(0, 2, 3))

# Calculate the mean values of validation images and masks
valid_images_mean = torch.mean(valid_images, dim=(0, 2, 3))
valid_masks_mean = torch.mean(valid_masks, dim=(0, 2, 3))

# Define the transformations for training images and masks
#train_transform = transforms.Compose([
 #   transforms.ToPILImage(),  # Convert the tensor image to a PIL image
  #  transforms.Resize((256, 256)),  # Resize image to a fixed size (e.g., 256x256)
   # transforms.ToTensor(),
    #transforms.Normalize(train_images_mean, [1.0, 1.0, 1.0]),  # Use the mean values of training images
#])



#train_mask_transform = transforms.Compose([
 #   transforms.ToPILImage(),
 #   transforms.Resize((256, 256)),
 #   transforms.ToTensor(),
 #   transforms.Normalize(train_masks_mean, [1.0]),  # Use the mean value of training masks (gray scale)
#])

# Define the transformations for validation images and masks
#val_transform = transforms.Compose([
 #   transforms.ToPILImage(),
 #   transforms.Resize((256, 256)),
 #   transforms.ToTensor(),
 #   transforms.Normalize(valid_images_mean, [1.0, 1.0, 1.0]),  # Use the mean values of validation images
#])

#val_mask_transform = transforms.Compose([
 #   transforms.ToPILImage(),
 #   transforms.Resize((256, 256)),
 #   transforms.ToTensor(),
 #   transforms.Normalize(valid_masks_mean, [1.0]),  # Use the mean value of validation masks (gray scale)
#])*/

In [ ]:

import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


from PIL import Image
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
import pdb


class COVERAGE(Dataset):
    def __init__(self, image_dir, mask_dir, transform =None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        #self.mask_transform = mask_transform
        self.images = os.listdir(image_dir)


    def __len__(self):
        return len(self.images)


    def __getitem__(self,index):
        #img_path = os.path.join(self.image_dir, self.images[index])
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index])

        #print(os.listdir(mask_path))
        #pdb.set_trace()

        image = np.array(Image.open(img_path).convert("RGB"))
        mask  = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask





In [ ]:

import torch
import torchvision




def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):

    train_ds = COVERAGE(
       image_dir=train_dir,
       mask_dir=train_maskdir,
       transform=train_transform,
    )

    train_loader = DataLoader(
       train_ds,
       batch_size=batch_size,
       num_workers=num_workers,
       pin_memory=pin_memory,
       shuffle=True,
    )

    val_ds = COVERAGE(
       image_dir=val_dir,
       mask_dir=val_maskdir,
       transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader


In [ ]:
def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0.0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            p1, p2 = model(x)
            preds = torch.sigmoid(p2)
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )

    print(
        f"Dice score: {dice_score/len(loader)}"
    )

In [ ]:

import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 20
NUM_WORKERS = 2
IMAGE_HEIGHT = 256 #1280
IMAGE_WIDTH = 256  #1918
PIN_MEMORY = True
LOAD_MODEL = True
TRAIN_IMG_DIR = "/content/drive/MyDrive/dataset/COVERAGE/train/images/"
TRAIN_MASK_DIR = "/content/drive/MyDrive/dataset/COVERAGE/train/masks/"
VAL_IMG_DIR = "/content/drive/MyDrive/dataset/COVERAGE/valid/images/"
VAL_MASK_DIR = "/content/drive/MyDrive/dataset/COVERAGE/valid/masks/"




def train_fn(loader, model, optimizer, loss_fn, scaler):
    model.train()  # Set model to training mode
    total_loss = 0.0
    all_predictions = []

    for batch_idx, (data, targets) in enumerate(loader):
        data = data.to(device=DEVICE)
        targets = targets.unsqueeze(1).to(device=DEVICE)
        print("Target type:", type(targets))

        # Print the statement and the shape of data
        #print(f"Shape of data: {data.shape}")
        # Print the statement and the shape of targets
        #print(f"Shape of targets: {targets.shape}")

        # Forward pass
        with torch.cuda.amp.autocast():
            predictions, pred = model(data)
            print("prediction type:", type(predictions))

            print(predictions.shape)
            print(pred.shape)

            #all_predictions.append(predictions)
            #print("Shape of all predictions:", all_predictions.shape)
            print("Shape of targets:", targets.shape)
            loss = loss_fn(pred, targets)



        #pdb.set_trace()
        #all_predictions = tuple(torch.cat(tensors, dim=0) for tensors in zip(*all_predictions))
        #targets = torch.cat(targets, dim=0)
        '''
        for i, prediction in enumerate(all_predictions):
          loss = loss_fn(prediction, targets[i])
        '''
        # Backward pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # Accumulate loss
        total_loss += loss.item()

        # Print progress
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}/{len(loader.dataset)} Loss: {loss.item():.4f}")

    # Calculate average loss
    avg_loss = total_loss / len(loader)
    print(f"Epoch Loss: {avg_loss:.4f}")




In [ ]:

from albumentations import Compose, Normalize, Resize, Rotate, HorizontalFlip, VerticalFlip
from albumentations.pytorch import ToTensorV2




def main():
    train_transform = A.Compose(
        [
           A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
           A.Rotate(limit=35, p=1.0),
           A.HorizontalFlip(p=0.5),
           A.VerticalFlip(p=0.1),

           A.Normalize(
               mean=[0.0, 0.0, 0.0],
               std=[1.0, 1.0, 1.0],
               max_pixel_value=255.0,
           ),
           ToTensorV2(),
        ],
        is_check_shapes=False
    )


    val_transforms = A.Compose(
        [
           A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
           A.Normalize(
               mean=[0.0, 0.0, 0.0],
               std=[1.0, 1.0, 1.0],
               max_pixel_value=255.0,
           ),
           ToTensorV2(),
        ],
        is_check_shapes=False
    )

    model = get_mvss(backbone='resnet50', pretrained_base=True, nclass=1, sobel=True, n_input=3, constrain=True)
    model = model.to(DEVICE)

    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

    check_accuracy(val_loader, model, device=DEVICE)



In [ ]:
if __name__ == "__main__":
    main()


load pretrain success
----------use sobel-------------
----------use constrain-------------
load pretrain success


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Target type: <class 'torch.Tensor'>
actual shape check: torch.Size([16, 3, 256, 256])
device of x: cpu


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256])
prediction type: <class 'torch.Tensor'>
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256])
Shape of targets: torch.Size([16, 1, 256, 256])
Batch 0/80 Loss: 1.3742
Target type: <class 'torch.Tensor'>
actual shape check: torch.Size([16, 3, 256, 256])
device of x: cpu
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256])
prediction type: <class 'torch.Tensor'>
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256])
Shape of targets: torch.Size([16, 1, 256, 256])
Target type: <class 'torch.Tensor'>
actual shape check: torch.Size([16, 3, 256, 256])
device of x: cpu
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256])
prediction type: <class 'torch.Tensor'>
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256])
Shape of targets: torch.Size([16, 1, 256, 256])
Target type: <class 'torch.Tensor'>
actual shape check: torch.Size([16, 3, 256, 256])
device of x: cpu
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 256, 256